# Sistem Diagnosis Penyakit dengan Gejala Tumpang Tindih
---

In [1]:
# !pip install pytholog pandas

import pandas as pd
import pytholog as pl
import numpy as np

In [2]:
# Memuat dataset
file_path = 'gejala_penyakit.csv'
df = pd.read_csv(file_path)

# Membersihkan nama kolom dari spasi yang tidak sengaja
df.columns = df.columns.str.strip()

# Fungsi untuk membersihkan dan mengubah data menjadi format pytolog
def clean_format(text):
    if pd.isna(text):
        return None
    return str(text).strip().replace(' ', '_')

# Membersihkan semua data teks di dataframe
df['penyakit'] = df['penyakit'].apply(clean_format)
gejala_cols = ['gejala1', 'gejala2', 'gejala3']
for col in gejala_cols:
    df[col] = df[col].apply(clean_format)

penyakit_symptom_map = {}
for index, row in df.iterrows():
    penyakit = row['penyakit']
    gejala_list = [g for g in [row['gejala1'], row['gejala2'], row['gejala3']] if g is not None]
    penyakit_symptom_map[penyakit] = gejala_list

print("Struktur data 'penyakit_symptom_map' berhasil dibuat.")
df.head(50)

Struktur data 'penyakit_symptom_map' berhasil dibuat.


,penyakit,gejala1,gejala2,gejala3,kepastian
0,flu,suhu_tubuh_naik,batuk,lemas,1.0
1,demam_berdarah,demam_tinggi,nyeri_otot,ruam_kulit,1.0
2,campak,mata_merah,ruam_di_wajah,batuk_ringan,1.0
3,alergi,sering_bersin,mata_gatal,ingusan,1.0
4,hipotiroid,sering_lelah,rambut_rontok,merasa_kedinginan,1.0
5,covid19,tidak_bisa_cium_bau,batuk_ringan,lelah_banget,1.0
6,asma,napas_bunyi,sesak_nafas,batuk_malam_hari,1.0
7,tuberkulosis,batuk_lama,keringat_malam,kurus_mendadak,1.0
8,bronkitis,batuk_berdahak,dada_sesak,capek_terus,1.0
9,pneumonia,napas_pendek,sakit_dada,susah_tidur_karena_sesak,1.0


In [3]:
def analisis_diagnosis(gejala_pasien, db_penyakit):

    # Bersihkan input gejala pasien
    gejala_pasien_set = set([clean_format(g) for g in gejala_pasien])
    
    hasil_analisis = []

    # Iterasi melalui setiap penyakit dalam database kita
    for penyakit, gejala_db_list in db_penyakit.items():
        gejala_db_set = set(gejala_db_list)
        
        # Cari irisan (gejala yang cocok) antara input pasien dan database
        gejala_cocok = gejala_pasien_set.intersection(gejala_db_set)
        
        # Hanya proses jika ada minimal satu gejala yang cocok
        if gejala_cocok:
            # Hitung persentase kecocokan
            total_gejala_db = len(gejala_db_set)
            kecocokan_persen = (len(gejala_cocok) / total_gejala_db) * 100
            
            # Cari gejala yang kurang (yang ada di DB tapi tidak diinput pasien)
            gejala_kurang = gejala_db_set - gejala_pasien_set
            
            # Simpan hasilnya
            hasil_analisis.append({
                'penyakit': penyakit,
                'kecocokan': kecocokan_persen,
                'gejala_cocok': list(gejala_cocok),
                'saran_gejala': list(gejala_kurang)
            })
            
    # Urutkan hasil dari kecocokan tertinggi ke terendah
    hasil_analisis_sorted = sorted(hasil_analisis, key=lambda x: x['kecocokan'], reverse=True)
    
    return hasil_analisis_sorted

In [5]:
# VVV UBAH GEJALA PASIEN DI BAWAH INI VVV
gejala_yang_dialami = ['lemas', 'ruam kulit', 'demam'] 
# ^^^ UBAH GEJALA PASIEN DI ATAS INI ^^^

print(f"Mencari diagnosis untuk gejala: {', '.join(gejala_yang_dialami)}\n")
hasil_diagnosis = analisis_diagnosis(gejala_yang_dialami, penyakit_symptom_map)

# Tampilkan hasil diagnosis sesuai format yang diminta
if hasil_diagnosis:
    print("Hasil Diagnosis:")
    for hasil in hasil_diagnosis:
        nama_penyakit = hasil['penyakit'].replace('_', ' ').title()
        kecocokan = hasil['kecocokan']
        saran = hasil['saran_gejala']
        
        # Cetak baris utama
        print(f"- {nama_penyakit} (Kecocokan: {kecocokan:.0f}%)")
        
        # Cetak saran gejala tambahan jika ada
        if saran:
            saran_formatted = ', '.join(saran).replace('_', ' ')
            print(f"  ↳ Saran gejala tambahan yang perlu ditanyakan: {saran_formatted}")
else:
    print(">>> Tidak ditemukan diagnosis yang cocok dengan gejala yang diberikan. <<<")

Mencari diagnosis untuk gejala: lemas, ruam kulit, demam

Hasil Diagnosis:
- Flu (Kecocokan: 33%)
  ↳ Saran gejala tambahan yang perlu ditanyakan: batuk, suhu tubuh naik
- Demam Berdarah (Kecocokan: 33%)
  ↳ Saran gejala tambahan yang perlu ditanyakan: demam tinggi, nyeri otot
